<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# 少样本提示

In [ ]:
from videos.walkthroughs import walkthrough_32 as walkthrough

In [ ]:
walkthrough()

在这个 notebook 中，您将学习一种叫做少样本提示的技术，向聊天模型提供指导性示例。

---

## 目标

完成这个 notebook 时，您将会：

- 理解如何执行少样本提示。
- 观察少样本提示技术的效果和局限性。
- 学习有效创建和编辑少样本提示的方法。

---

## 导入

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

---

## 创建模型实例

In [ ]:
base_url = 'http://llama:8000/v1'
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(base_url=base_url, model=model, temperature=0)

---

## 少样本提示

我们可以提供人类与 AI 的示例交互作为提示的一部分，以影响模型的响应行为。我们来创建一个简单的链开始探索这个思路。

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("human", "{prompt}")
])

In [ ]:
parser = StrOutputParser()

In [ ]:
chain = prompt_template | llm | parser

假设出于某种原因，我们希望模型将我们说的内容以大写字母重复给我们。可以这样做。

In [ ]:
chain.invoke({"prompt": "Repeat back whatever I say to you, but in all capital letters: hello"})

这个提示词中的明确指令足以让模型像我们希望的那样作出响应。不过，让我们也尝试通过少样本提示让模型实现相同的行为。为此，我们将扩展提示模板，包含多个人类与 AI 的示例交互，而不依赖于给模型明确的指令。

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("human", "hello"),
    ("ai", "HELLO"),
    ("human", "red"),
    ("ai", "RED"),
    ("human", "blue"),
    ("ai", "BLUE"),
    ("human", "{prompt}")
])

In [ ]:
chain = prompt_template | llm | parser

构造完少样本提示后，我们再给模型发一个新提示词，但这次不提供任何关于模型应该做什么的明确指令。

In [ ]:
chain.invoke({"prompt": "hi"})

In [ ]:
chain.invoke({"prompt": "orange"})

有趣的是，我们在任何时候都没有给模型提供任何明确或具体的指令，告诉它该做什么。

那么，这些少样本提示示例到底会对模型产生多大影响呢？我们得测一下才知道。

这里有一个例子，我们得到了大写字母，但单词却变了。

In [ ]:
chain.invoke({"prompt": "nvidia"})

由于所有的示例都是单个词单响应的例子，如果与示例不同的提示词没按预期响应，也不应该感到太惊讶。

In [ ]:
chain.invoke({"prompt": "How's it going today?"})

---

## 附注：少样本提示单独使用

为了学习核心概念，我们在这里单独介绍了少样本提示，但实际上，少样本提示只是您在使用 LLM 时可用的几种工具之一，通常会与其它技术结合，比如在主提示词中具体说明（我们已经介绍过）和利用系统消息，这将在下一个 notebook 中讨论。

现在我们先继续探索少样本提示的应用，主要是为让您在结合其它技术之前，专注于学习如何熟练地使用它。

---

## 少样本提示模板

在我们希望按少量示例进行后续交互时，将这些少样本提示硬编码到一个提示模板中没什么问题，正如我们刚刚做的那样。

不过，我们有时会希望用（或尝试）不同的示例组合，配合一个静态的基础提示词。或者从数据中获取用户/AI的示例交互。这种情况下，可以使用 `FewShotChatMessagePromptTemplate`。

假设我们想构建一个提示词，用于生成关于某个城市的信息，但对模型生成的信息格式有非常具体的要求。这听起来是少样本提示的一个不错的用例。

最终提示可能会像下面这样，带有一个供人类使用的城市名称的提示模板，但还有一个（目前已注释掉，因为尚未定义）包含模型应如何格式化其响应的示例的 `few_shot_prompt`。

In [ ]:
city_info_prompt_template = ChatPromptTemplate.from_messages([
    # few_shot_prompt, # NOTE: we would like to provide several examples here in the form of a few-shot prompt.
    ("human", "Provide information about the following city in exactly the same format as you've done in previous responses: City: {city}")
])

无论是自己编写，还是从数据源获取，亦或是让 LLM 为我们生成，我们都可以构建一个人类/AI交互示例的列表，以便在少样本提示中使用。

In [ ]:
city_examples_location = [
    {"city": "Oakland", "output": "Oakland, USA, North America, Earth"},
    {"city": "Paris", "output": "Paris, France, Europe, Earth"},
    {"city": "Lima", "output": "Lima, Peru, South America, Earth"},
    {"city": "Seoul", "output": "Seoul, South Korea, Asia, Earth"}
]

由于 `city_examples_location` 中的每项都代表了一条人类消息和一条 AI 消息，所以有必要构建一个提示模板来表示单个示例交互。

In [ ]:
prompt_template_for_examples = ChatPromptTemplate.from_messages([
    ("human", "{city}"),
    ("ai", "{output}"),
])

现在，结合我们的示例和用于示例的提示模板，可以用 `FewShotChatMessagePromptTemplate` 从示例中构建一个少样本提示。

`FewShotChatMessagePromptTemplate` 需要两个参数：
1. `examples`：一个字典列表（显然包含我们的示例）
2. `example_prompt`：用于构建示例的提示模板（显然分为 `human` 和 `ai` 消息）

In [ ]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=city_examples_location,
    example_prompt=prompt_template_for_examples
)

`few_shot_prompt` 是一个运行时，就像其它任何提示模板一样，我们可以调用它。

In [ ]:
few_shot_prompt.invoke({}).to_messages()

定义了 `few_shot_prompt` 之后，就可以重新审视 `city_info_prompt_template`，它实际上应该包含 `few_shot_prompt`。

In [ ]:
city_info_prompt_template = ChatPromptTemplate.from_messages([
    few_shot_prompt,
    ("human", "Provide information about the following city in exactly the same format as you've done in previous responses: City: {city}")
])

最后，我们可以创建一个链并使用它，这里发一批城市名称给它。

In [ ]:
chain = city_info_prompt_template | llm | parser

In [ ]:
cities = [
    "New York",
    "London",
    "Tokyo",
    "Sydney",
    "Cape Town",
    "Toronto",
    "Berlin",
    "Buenos Aires",
    "Dubai",
    "Singapore"
]

In [ ]:
chain.batch(cities)

您可能觉得为了放一些示例到提示词中我们做了很多工作，您或许只想把示例硬编码到主提示词中。不过，正如我们之前提到的，有些场景您可能希望在保持主提示词其余结构的同时，替换示例，这个方法就恰好可以做到。

---

## 练习：替换不同的少样本示例

这个练习中，您将重新创建我们刚刚做的链，但要用下面提供的一组新的少样本示例。

我们还将提供其余链的代码，您的任务就是**尽量少修改以使其适用于新示例**。

如果您遇到困难，可以随时查看下面的*参考答案*。

### 您的代码

In [ ]:
city_examples_slogan = [
    {"city": "Oakland", "output": "Oakland, Bright Side of the Bay"},
    {"city": "Paris", "output": "Paris, City of Light"},
    {"city": "Lima", "output": "Lima, City of Kings"},
    {"city": "Seoul", "output": "Seoul, Heartbeat of Korea"},
]

In [ ]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=city_examples_location,
    example_prompt=prompt_template_for_examples
)

In [ ]:
city_info_prompt_template = ChatPromptTemplate.from_messages([
    few_shot_prompt,
    ("human", "Provide information about the following city in exactly the same format as you've done in previous responses: City: {city}")
])

In [ ]:
chain = city_info_prompt_template | llm | parser

In [ ]:
chain.batch(cities)

### 参考答案

In [ ]:
city_examples_slogan = [
    {"city": "Oakland", "output": "Oakland, Bright Side of the Bay"},
    {"city": "Paris", "output": "Paris, City of Light"},
    {"city": "Lima", "output": "Lima, City of Kings"},
    {"city": "Seoul", "output": "Seoul, Heartbeat of Korea"},
]

**我们对之前的链所做的唯一更改就在这，将 `examples` 字段更新为新的 `city_examples_slogan` 列表。**

In [ ]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=city_examples_slogan, # Only change made is right here.
    example_prompt=prompt_template_for_examples
)

In [ ]:
city_info_prompt_template = ChatPromptTemplate.from_messages([
    few_shot_prompt,
    ("human", "Provide information about the following city in exactly the same format as you've done in previous responses: City: {city}")
])

In [ ]:
chain = city_info_prompt_template | llm | parser

In [ ]:
chain.batch(cities)

---

## 总结

在这个 notebook 中，您使用人类和 AI 消息组合出了示例交互，从而影响了 LLM 行为。

下一个 notebook 您将学习第三种消息类型，即系统消息，并用它来为聊天模型建立一个整体的人设和角色。